In [1]:
from functools import partial
import ipyvuetify as v
import ee

In [2]:
ee.Initialize()

In [3]:
# Get all columns from asset
# Let the user select which is the LC/LU column, could be just code or with description
# Ask user if there is a description column
# if there is a description, load both columns
# From all the classes let the user to choose the corresponding ESA class


In [4]:
aoi_path = 'users/dafguerrerom/FAO/LULC_2012_AOI'

In [5]:
asset = ee.FeatureCollection(aoi_path)

In [6]:
def get_fields(column):
    return sorted([str(e) for e in list(set(asset.aggregate_array(column).getInfo()))])

In [7]:
def get_cols(asset):
    columns = ee.Feature(asset.first()).propertyNames().getInfo()
    return sorted([str(col) for col in columns if col not in ['system:index', 'Shape_Area']])

In [8]:
from traitlets import Dict, HasTraits

In [9]:
cols = get_cols(asset)
cols;

In [16]:
code_field = get_fields('CODIGO')[:5]
desc_field = get_fields('LEYENDA3N')[:5]

In [78]:
class Matrix(v.SimpleTable):
    
    matrix = Dict({}).tag(sync=True)
    
    def __init__(self, code_field, desc_field):
        
        self.code_field = code_field
        self.desc_field = desc_field
        self.desc_code = list(zip(self.code_field, self.desc_field))
        
        headers = ['Code', 'Description', 'Ipcc']
        
        # Instantiate an empty dictionary with code as keys and empty values
        for code in self.code_field:
            self.matrix[code] = ''
        
        header = [
            v.Html(
                tag = 'tr', 
                children = (
                    [v.Html(tag = 'th', children = [h]) for h in headers]
                )
            )
        ]
        
        rows = [
            v.Html(tag='tr', children=[
                v.Html(tag = 'td', children=[code]), 
                v.Html(tag = 'td', children=[desc]),
                self.get_ipcc_class(code),
                
            ]) for code, desc in self.desc_code
        ]
        
        self.children = [v.Html(tag = 'tbody', children = header + rows)]
        
        # Create table
        super().__init__()
    
    def store(self, code, change):
        self.matrix[code] = change['new']
        
    def get_ipcc_class(self, code):
        
        """Get a selector """

        select = v.Select(_metadata={'name':code}, items=[1,2,3,4,5,6], v_model=None)
        select.observe(partial(self.store, code), 'v_model')
        
        return select
        

In [79]:
transition = Matrix(code_field, desc_field)

In [80]:
transition

Matrix(children=[Html(children=[Html(children=[Html(children=['Code'], tag='th'), Html(children=['Description'…